# Cat Skin Disease Classification using CNN (Transfer Learning)

Notebook ini berisi implementasi machine learning untuk mengklasifikasikan penyakit kulit pada kucing menggunakan metode Convolutional Neural Network (CNN). Kita akan menggunakan **Transfer Learning** dengan MobileNetV2 yang merupakan best practice untuk mendapatkan akurasi tinggi dengan dataset terbatas.

**Dataset Path (Google Drive):** `/content/drive/MyDrive/CATSCKIN/CAT SKIN DISEASE/`

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib

# Konfigurasi GPU (jika tersedia)
try:
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU Detected: {len(gpus)}")
    else:
        print("No GPU detected, running on CPU")
except Exception as e:
    print(f"Error configuring GPU: {e}")

## 1. Persiapan Dataset

Kita akan memuat dataset langsung dari Google Drive. Dataset akan dibagi menjadi:
- **Training Set (80%)**: Untuk melatih model.
- **Validation Set (20%)**: Untuk evaluasi saat training agar tidak overfitting.

In [ ]:
# Lokasi dataset di Google Drive
data_dir = pathlib.Path('/content/drive/MyDrive/CATSCKIN/CAT SKIN DISEASE/')

# Cek apakah direktori dataset ditemukan
if not data_dir.exists():
    print(f"WARNING: Dataset tidak ditemukan di {data_dir}. Pastikan path sudah benar.")
else:
    print(f"Dataset ditemukan di {data_dir}")

# Parameter Loader
batch_size = 32
img_height = 224  # MobileNetV2 menggunakan 224x224
img_width = 224

print("Training Data:")
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

print("\nValidation Data:")
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(f"\nKelas yang ditemukan: {class_names}")

## 2. Visualisasi Data

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")
plt.show()

## 3. Optimasi Performa (Prefetching)
Menggunakan `cache()` dan `prefetch()` agar loading data tidak menjadi bottleneck saat training.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## 4. Data Augmentation
Karena dataset mungkin terbatas, kita melakukan augmentasi (memutar, membalik, zoom) secara acak pada gambar input untuk mencegah overfitting.

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

## 5. Membangun Model (Transfer Learning)

Kita menggunakan **MobileNetV2** yang sudah dilatih dengan ImageNet sebagai base model. Ini akan mempercepat training dan meningkatkan akurasi.

1.  **Rescaling**: Normalisasi pixel ke [-1, 1] (sesuai kebutuhan MobileNetV2).
2.  **Base Model**: MobileNetV2 (tanpa top layer).
3.  **Classification Head**: GlobalAveragePooling2D + Dense Output Layer.

In [ ]:
num_classes = len(class_names)

# Menggunakan pre-trained model MobileNetV2
base_model = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),
                                               include_top=False,
                                               weights='imagenet')

# Bekukan base model agar bobotnya tidak berubah di awal training
base_model.trainable = False

model = Sequential([
  data_augmentation,
  layers.Rescaling(1./127.5, offset=-1), # Preprocessing khusus MobileNetV2
  base_model,
  layers.GlobalAveragePooling2D(),
  layers.Dropout(0.2), # Mencegah overfitting
  layers.Dense(num_classes, activation='softmax') # Output layer
])

model.summary()

## 6. Compile dan Training Model

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

epochs = 15

# Callback untuk menyimpan model terbaik dan menghentikan training jika tidak ada perbaikan
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint('best_model.keras', save_best_only=True)
]

history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  callbacks=callbacks
)

## 7. Evaluasi Hasil Training

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## 8. Fine Tuning (Opsional - Tahap Lanjutan)
Jika akurasi masih kurang, kita bisa melakukan fine-tuning dengan membuka beberapa layer atas dari base model untuk dilatih kembali dengan learning rate yang sangat kecil.

In [ ]:
# Unfreeze base model
base_model.trainable = True

# Fine-tune dari layer ke-100 ke atas
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Compile ulang dengan learning rate rendah
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5),
              metrics=['accuracy'])

print("Mulai Fine-Tuning...")
history_fine = model.fit(train_ds,
                         epochs=10, # Tambahan epoch
                         initial_epoch=history.epoch[-1],
                         validation_data=val_ds,
                         callbacks=callbacks)

## 9. Prediksi Gambar Baru
Contoh cara menggunakan model untuk memprediksi gambar baru.

In [ ]:
# Contoh prediksi (ambil 1 gambar dari val_ds)
for images, labels in val_ds.take(1):
    img_array = images[0].numpy()
    actual_label = class_names[labels[0]]
    
    img_batch = np.expand_dims(img_array, 0)
    predictions = model.predict(img_batch)
    score = tf.nn.softmax(predictions[0])

    plt.imshow(img_array.astype("uint8"))
    plt.title(f"Actual: {actual_label}\nPred: {class_names[np.argmax(score)]} ({100 * np.max(score):.2f}%)")
    plt.axis("off")
    plt.show()

In [ ]:
# Simpan model final
model.save('cat_skin_disease_model_final.h5')
print("Model berhasil disimpan!")